# In this note book, we're going to cover some of the most fundamental concepts of tensors usning TensorFlow

More sprecificlly, we are going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function( a way to speed up your regular Python Function )
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!

## Introduction to Tensors

In [1]:
# Import Tensorflow
import tensorflow as tf

print(tf.__version__)

2.11.0


In [2]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-19 06:31:39.389495: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-19 06:31:39.389800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# check the number of dimansions of a tensor (ndim stands for number dimansion)
scalar.ndim

0

In [4]:
# create vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# check dimansion
vector.ndim

1

In [9]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [10]:
matrix.ndim

2

In [13]:
# Create a tensor (has more than 2 dimension)
tensors = tf.constant([[[1,2,3]],
                      [[4,5,6]],
                      [[7,8,9]],
                      [[10,11,12]]])
tensors

<tf.Tensor: shape=(4, 1, 3), dtype=int32, numpy=
array([[[ 1,  2,  3]],

       [[ 4,  5,  6]],

       [[ 7,  8,  9]],

       [[10, 11, 12]]], dtype=int32)>

In [14]:
tensors.ndim

3

what we have created so far:
* Scalar: a single number
* Vector: a number with direction (e.g wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimentional array of numbers

#### Creating tesnors with `tf.Variable`

In [15]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [16]:
# lets try change 
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [17]:
# how about if try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [18]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

### Creating random tensors

Random tensors are tensors of some abitrary size which contain random numbers.

In [19]:
# Create two random tensors
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.75658023, -0.06854693],
       [ 0.07595028, -1.2573844 ],
       [-0.23193759, -1.8107857 ]], dtype=float32)>

In [21]:
# Create two random tensors
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-1.3240397 ,  0.28785658],
       [-0.87579006, -0.08856997],
       [ 0.6921164 ,  0.842157  ]], dtype=float32)>

In [22]:
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False, False],
       [False, False],
       [False, False]])>

In [55]:
tf.random.set_seed(42)
fixed_ts = tf.random.shuffle(tensors)
fixed_ts

<tf.Tensor: shape=(4, 1, 3), dtype=int32, numpy=
array([[[10, 11, 12]],

       [[ 1,  2,  3]],

       [[ 4,  5,  6]],

       [[ 7,  8,  9]]], dtype=int32)>

### Indexing tensors
Tensors can be indexed just like python lists.

In [54]:
# Get the 2 first 2 elemnts of each dimension
fixed_ts[:1, :3, :2]

<tf.Tensor: shape=(1, 1, 2), dtype=int32, numpy=array([[[10, 11]]], dtype=int32)>

In [57]:
fixed_ts.shape

TensorShape([4, 1, 3])

In [58]:
fixed_ts[..., tf.newaxis].shape

TensorShape([4, 1, 3, 1])

In [59]:
import numpy as np

narray = np.array([10,20,30])
narray

array([10, 20, 30])

In [60]:
narray+10

array([20, 30, 40])

In [61]:
tf_array = tf.constant([10,20,30])
tf_array

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([10, 20, 30], dtype=int32)>

In [62]:
tf_array+10

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([20, 30, 40], dtype=int32)>

In [63]:
tf_array*2

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([20, 40, 60], dtype=int32)>

In [65]:
X = tf.constant([[1,2],
                 [4,5],
                 [7,8]])

Y = tf.constant([[10,11],
                 [13,14],
                 [16,17]])

X.shape, Y.shape

(TensorShape([3, 2]), TensorShape([3, 2]))

In [66]:
tf.matmul(X, Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [71]:
X_n = np.array([[1,2],
                 [4,5],
                 [7,8]])

Y_n = np.array([[10,11],
                 [13,14],
                 [16,17]])
Y_n.shape, Y_n.shape

((3, 2), (3, 2))

In [72]:
np.dot(X_n, Y_n)

ValueError: shapes (3,2) and (3,2) not aligned: 2 (dim 1) != 3 (dim 0)

In [73]:
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[10, 11, 13],
       [14, 16, 17]], dtype=int32)>

In [74]:
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 38,  43,  47],
       [110, 124, 137],
       [182, 205, 227]], dtype=int32)>

In [75]:
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100, 107],
       [269, 289]], dtype=int32)>

In [76]:
# check the values of Y, reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2,3):")
print(tf.reshape(Y, (2,3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[10 11]
 [13 14]
 [16 17]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3):
tf.Tensor(
[[10 11 13]
 [14 16 17]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[10 13 16]
 [11 14 17]], shape=(2, 3), dtype=int32)
